In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import pandas as pd
import bokeh
import bokeh.plotting
import bokeh.palettes
import bokeh.models
import scipy
from pathlib import Path

import hvplot.pandas

import stochastic_sc_model as ssc_model

# Ensure we don't hit floating point errors
np.seterr('raise')
np.seterr(under='ignore')

colors20 = bokeh.palettes.d3['Category20'][20]

In [ ]:
# Load generated dataframes
summary_data = pd.concat([pd.read_feather(f) for f in Path('output/dataframes').glob('worker*.feather')])
summary_data['fano'] = summary_data['A_std']**2 / summary_data['A_mean']

In [ ]:
fano_data = summary_data[(summary_data['topo_rate']==1/1200) & (summary_data['B_promoter_strength']==1) & (summary_data['spacing']==400)]

In [ ]:
fano_data

In [ ]:
fano_data['A_std'] ** 2 / fano_data['A_mean']

In [ ]:
summary_data['topo_rate'].unique()

In [ ]:
summary_data[(summary_data['geometry']=='tandem') & (summary_data['topo_rate']==1/1200)].hvplot.scatter(x='B_promoter_strength', y='A_mean', color='spacing', cmap='viridis')

In [ ]:
summary_data[summary_data['B_promoter_strength']==1].hvplot.scatter(x='spacing', y='fano', by=['topo_rate'], color='geometry', subplots=True).cols(2)

In [ ]:
summary_data.hvplot.scatter(x='B_promoter_strength', y='A_mean', by=['topo_rate', 'geometry'], color='spacing', cmap='viridis', subplots=True).cols(3)

In [ ]:
summary_data.topo_rate

In [ ]:
averaged_data = summary_data.groupby(['geometry', 'B_promoter_strength', 'spacing']).mean().reset_index()

In [ ]:
simplified_data = [averaged_data[(averaged_data['geometry']==geom) & (averaged_data['spacing']==400)]
                   for geom in ['tandem', 'divergent', 'convergent']]
for df in simplified_data:
    scale_factor = 1.0 / df[df['B_promoter_strength']==.01]['A_mean'].iloc[0]
    df['A_mean'] *= scale_factor
    df['A_std'] *= scale_factor

In [ ]:
simplified_data[0]

In [ ]:
tandem_line, = plt.plot(simplified_data[0]['B_promoter_strength'], simplified_data[0]['A_mean'], color='#272060', linewidth=4)
div_line,    = plt.plot(simplified_data[1]['B_promoter_strength'], simplified_data[1]['A_mean'], color='#17b8cf', linewidth=4)
conv_line,   = plt.plot(simplified_data[2]['B_promoter_strength'], simplified_data[2]['A_mean'], color='#ed3a38', linewidth=4)
tandem_fill = plt.fill_between(simplified_data[0]['B_promoter_strength'],
                                simplified_data[0]['A_mean'] - simplified_data[0]['A_std'] * .5,
                                simplified_data[0]['A_mean'] + simplified_data[0]['A_std'] * .5,
                                alpha=.1, color='#272060')
div_fill    = plt.fill_between(simplified_data[1]['B_promoter_strength'],
                                simplified_data[1]['A_mean'] - simplified_data[1]['A_std'] * .5,
                                simplified_data[1]['A_mean'] + simplified_data[1]['A_std'] * .5,
                                alpha=.1, color='#17b8cf')
conv_fill   = plt.fill_between(simplified_data[2]['B_promoter_strength'],
                                simplified_data[2]['A_mean'] - simplified_data[2]['A_std'] * .5,
                                simplified_data[2]['A_mean'] + simplified_data[2]['A_std'] * .5,
                                alpha=.1, color='#ed3a38')
plt.ylabel('Reporter output')
plt.xlabel('Induction level')
plt.xlim([0,1])
plt.rcParams.update({'font.family': 'Arial', 'font.size': 16})
plt.legend([(tandem_line, tandem_fill), (div_line, div_fill), (conv_line, conv_fill)],
           ['Tandem','Divergent','Convergent'],
           ncol=3, loc='upper center')
plt.tick_params(axis='both', which='both', bottom=True, top=True, labelbottom=True, right=True, left=True, labelleft=True) 
plt.savefig('reporter_behavior.svg')
plt.show()

In [ ]:
divergent_data = [averaged_data[(averaged_data['geometry']=='convergent') & (averaged_data['spacing'] == space)] for space in averaged_data['spacing'].unique()]
scale_factor = divergent_data[0][divergent_data[0]['B_promoter_strength']==.01]['A_mean'].iloc[0]
lines = LineCollection([np.column_stack([df['B_promoter_strength'], df['A_mean']]) for df in divergent_data], cmap='viridis')
lines.set_array(averaged_data['spacing'].unique())
lines.set_linewidth(4)
plt.rcParams.update({'font.family': 'Arial', 'font.size': 16})
fig, ax = plt.subplots()
ax.set_ylim(.4, 12)
ax.set_xlim(0, 1)
line_collection = ax.add_collection(lines)
fig.colorbar(line_collection, ax=ax, label='Δx (bp)')
plt.ylabel('Reporter output')
plt.xlabel('Induction level')
ax.tick_params(axis='both', which='both', bottom=True, top=False, labelbottom=True, right=False, left=True, labelleft=True)
fig.tight_layout()
plt.savefig('dx.svg')

In [ ]:
line_collection.get_color()

In [ ]:
conv_topo_data = summary_data[(summary_data['geometry']=='convergent') & (summary_data['spacing'] == 500)]
conv_topo_data = conv_topo_data.groupby(['topo_rate', 'B_promoter_strength']).mean().reset_index()
topo_rates = conv_topo_data['topo_rate'].unique()
split_topo_data = [conv_topo_data[conv_topo_data['topo_rate'] == rate] for rate in topo_rates]
lines = LineCollection([np.column_stack([df['B_promoter_strength'], df['A_mean'] / 8.8]) for df in split_topo_data], cmap='viridis')
lines.set_array(topo_rates / min(topo_rates))
lines.set_linewidth(4)

plt.rcParams.update({'font.family': 'Arial', 'font.size': 16})
fig, ax = plt.subplots()
ax.set_ylim(0, 1.1)
ax.set_xlim(0, 1)
line_collection = ax.add_collection(lines)
fig.colorbar(line_collection, ax=ax, label='Relative topoisomerase activity')
plt.ylabel('Reporter output')
plt.xlabel('Induction level')
ax.tick_params(axis='both', which='both', bottom=True, top=False, labelbottom=True, right=False, left=True, labelleft=True)
fig.tight_layout()
plt.savefig('topo_behavior.svg')
plt.show()

In [ ]:
div_topo_data = summary_data[(summary_data['geometry']=='divergent') & (summary_data['spacing'] == 500)]
div_topo_data = div_topo_data.groupby(['topo_rate', 'B_promoter_strength']).mean().reset_index()
topo_rates = div_topo_data['topo_rate'].unique()
split_topo_data = [div_topo_data[div_topo_data['topo_rate'] == rate] for rate in topo_rates]
lines = LineCollection([np.column_stack([df['B_promoter_strength'], df['A_mean'] / 8.8]) for df in split_topo_data], cmap='viridis')
lines.set_array(topo_rates / min(topo_rates))
lines.set_linewidth(4)

plt.rcParams.update({'font.family': 'Arial', 'font.size': 16})
fig, ax = plt.subplots()
ax.set_ylim(0, 1.1)
ax.set_xlim(0, 1)
line_collection = ax.add_collection(lines)
fig.colorbar(line_collection, ax=ax, label='Relative topoisomerase activity')
plt.ylabel('Reporter output')
plt.xlabel('Induction level')
ax.tick_params(axis='both', which='both', bottom=True, top=False, labelbottom=True, right=False, left=True, labelleft=True)
fig.tight_layout()
plt.savefig('div_topo_behavior.svg')
plt.show()

In [ ]:
lines = LineCollection([np.column_stack([conv_topo_data[conv_topo_data['topo_rate']==rate]][])])
conv_topo_data = summary_data[(summary_data['geometry']=='convergent') & (summary_data['spacing'] == 500)]
tandem_line, = plt.plot(simplified_data[0]['B_promoter_strength'], simplified_data[0]['A_mean'], color='#272060', linewidth=4)
div_line,    = plt.plot(simplified_data[1]['B_promoter_strength'], simplified_data[1]['A_mean'], color='#17b8cf', linewidth=4)
conv_line,   = plt.plot(simplified_data[2]['B_promoter_strength'], simplified_data[2]['A_mean'], color='#ed3a38', linewidth=4)
tandem_fill = plt.fill_between(simplified_data[0]['B_promoter_strength'],
                                simplified_data[0]['A_mean'] - simplified_data[0]['A_std'] * .5,
                                simplified_data[0]['A_mean'] + simplified_data[0]['A_std'] * .5,
                                alpha=.1, color='#272060')
div_fill    = plt.fill_between(simplified_data[1]['B_promoter_strength'],
                                simplified_data[1]['A_mean'] - simplified_data[1]['A_std'] * .5,
                                simplified_data[1]['A_mean'] + simplified_data[1]['A_std'] * .5,
                                alpha=.1, color='#17b8cf')
conv_fill   = plt.fill_between(simplified_data[2]['B_promoter_strength'],
                                simplified_data[2]['A_mean'] - simplified_data[2]['A_std'] * .5,
                                simplified_data[2]['A_mean'] + simplified_data[2]['A_std'] * .5,
                                alpha=.1, color='#ed3a38')
plt.ylabel('Reporter output')
plt.xlabel('Induction level')
plt.xlim([0,1])
plt.rcParams.update({'font.family': 'Arial', 'font.size': 16})
plt.legend([(tandem_line, tandem_fill), (div_line, div_fill), (conv_line, conv_fill)],
           ['Tandem','Divergent','Convergent'],
           ncol=3, loc='upper center')
plt.tick_params(axis='both', which='both', bottom=True, top=True, labelbottom=True, right=True, left=True, labelleft=True) 
plt.savefig('reporter_behavior.svg')
plt.show()

In [ ]:
np.column_stack([divergent_data[1]['A_mean'], divergent_data[1]['B_promoter_strength']])

In [ ]:
summary_data.hvplot.scatter(x='A_mean', y='B_promoter_strength', color='spacing', cmap='viridis', by='geometry', subplots=True).cols(1)

In [ ]:
plt.plot()